In [2]:
import numpy as np
import pandas as pd
import pandas_profiling
from pandas.io.json import json_normalize 

import json
import logging

import psycopg2
from psycopg2.extras import Json

from sqlalchemy import create_engine, MetaData
import io

from input import json_schema

## Get dataset

In [3]:
with open("data/source.12-items.json") as f:
    d = json.load(f)

awhcuris = json_normalize(d)
address_data = awhcuris.head(31)
address_data.T

,0,1,2,3,4,5,6,7,8,9,10,11,12
_sync.flags,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28
_sync.history.bodymap.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""address"":{""barangay"":null,""country"":""Philipp..."
_sync.history.bodymap.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""address"":{""barangay"":"""",""country"":""Philippin..."
_sync.history.bodymap.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""address"":{""barangay"":""SvayPria"",""country"":""C..."
_sync.history.bodymap.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""address"":{""barangay"":"""",""country"":""Philippin..."
_sync.history.channels,[None],[None],"[None, None, None, None, None, None, None, Non...","[None, None, None, None, None]",[None],[None],"[None, None, None, None, None, None, None, Non...",[None],[None],[None],[None],"[None, None, None, None, None, None, None, Non...","[None, None, None, None, None, None, None, Non..."
_sync.history.parents,[-1],[-1],"[10, 5, 6, 2, 0, 4, 13, 9, 7, 3, -1, 1, 11, 12]","[4, 0, 1, 2, -1]",[-1],[-1],"[15, -1, 5, 2, 6, 1, 3, 11, 10, 14, 7, 0, 4, 1...",[-1],[-1],[-1],[-1],"[-1, 0, 7, 5, 2, 11, 9, 1, 14, 8, 6, 4, 3, 12,...","[34, 27, -1, 71, 58, 53, 4, 0, 28, 11, 72, 18,..."
_sync.history.revs,[1-96664e86d736f4c1ceaa5df1ccec808f],[1-efb2411a16e9c160e17cc648ab8f92a3],"[2-cb958bbc2ae7825f2c553d5550af2c59, 5-708c145...","[2-636a886e078abffcb7ae6f26ed4e3145, 3-8fe2be8...",[1-dafa2fc4b93a20d194602f2936fc459e],[1-2b67b5a8a3b6aeb0f28c2828ae2ce80e],"[12-d3adaa6a80ed2f1230d7cba436e4043f, 1-0c168e...",[1-3977141bbb0838b852bac4d8a8408324],[1-2c8b75610f7c4d2087c4bba404f8678c],[1-ac0d0b52fdd7b57feb0204254d147dae],[1-e5af0f925ce4d280fc4d193f740355f4],"[1-fe2026b4c541443ddacfaddc13964727, 2-0df63e9...","[40-bece4495cff1f73cd2075f907bf37ce8, 47-581af..."
_sync.new_rev,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18-ad8262d9d383cb894709e1a112293bb5
_sync.recent_sequences,[3001],[3136],[1988],[388],[3027],[2765],[352],[1592],[197],[3530],[2993],[847],"[1015, 1294, 1296, 1302, 1304, 1311, 1313, 132..."


In [341]:
category = 'child_healths'
row_data = awhcuris.loc[0][category]
child_data = json_normalize(row_data)
child_data = child_data.fillna(np.nan).replace('', np.nan,regex=True)
                                         
child_data = child_data.add_prefix(category + '.')
child_data['cb_id'] = awhcuris.loc[0]['cb_id']
child_data.T

,0
child_healths.child.eating_solid_food,False
child_healths.child.exclusive_breast_feeding,False
child_healths.child.following_experience,[]
child_healths.child.following_received,[]
child_healths.child.mid_upper_arm_circumference,[]
child_healths.date_updated,2018-09-21T12:50:16.184744
child_healths.infant.eating_solid_food,False
child_healths.infant.exclusive_breast_feeding,False
child_healths.infant.following_experience,[]
child_healths.infant.following_received,[]


In [345]:
category = 'symptoms_collection'
row_data = awhcuris.loc[2][category]
ss_data = json_normalize(row_data)
ss_data = ss_data.fillna(np.nan).replace('', np.nan,regex=True)

ss_data = ss_data.add_prefix(category + '.')
ss_data['cb_id'] = awhcuris.loc[2]['cb_id']
ss_data.T

,0
symptoms_collection.abdomen.abdomen,"[Muscle pain, weakness, or cramps, Lump or bul..."
symptoms_collection.arms.elbow,[]
symptoms_collection.arms.fingers,[]
symptoms_collection.arms.hand_and_palm,[]
symptoms_collection.arms.lower_arm,[]
symptoms_collection.arms.upper_arm,[]
symptoms_collection.arms.wrist,[]
symptoms_collection.chest.chest,[]
symptoms_collection.chest.lungs_and_breathing,[]
symptoms_collection.date_updated,09/20/2018 at 01:58:58 PM GMT+08:00


In [346]:
category = 'health_informations'
row_data = awhcuris.loc[9][category]
hi_data = json_normalize(row_data)
hi_data = hi_data.fillna(np.nan).replace('', np.nan,regex=True)

hi_data = hi_data.add_prefix(category + '.')
hi_data['cb_id'] = awhcuris.loc[9]['cb_id']
hi_data.T

,0
health_informations.alcohol_in_a_week,NaN
health_informations.allergies,NaN
health_informations.blood_pressure.first_reading.diastole,NaN
health_informations.blood_pressure.first_reading.systole,NaN
health_informations.blood_pressure.second_reading.diastole,NaN
health_informations.blood_pressure.second_reading.systole,NaN
health_informations.blood_sign,NaN
health_informations.blood_sugar,NaN
health_informations.blood_type,NaN
health_informations.date_updated,2018-10-09T15:30:45.873940


In [347]:
data = pd.DataFrame()
#data.append(hi_data).append(child_data).append(ss_data).T
#child_data.join(ss_data)

frames = [hi_data, ss_data, child_data]

result = hi_data.append(ss_data,sort=True)
result = result.append(child_data,sort=True)
result.T.head(1000)

,0,0,0
cb_id,d97b96af-112b-4e64-8a80-a8a0b2ea7dbd,00793b66-3367-4be3-8b67-d4b972db5c8d,000c8a6139cda10a4188d815c8dfc70f
child_healths.child.eating_solid_food,NaN,NaN,False
child_healths.child.exclusive_breast_feeding,NaN,NaN,False
child_healths.child.following_experience,NaN,NaN,[]
child_healths.child.following_received,NaN,NaN,[]
child_healths.child.mid_upper_arm_circumference,NaN,NaN,[]
child_healths.date_updated,NaN,NaN,2018-09-21T12:50:16.184744
child_healths.infant.eating_solid_food,NaN,NaN,False
child_healths.infant.exclusive_breast_feeding,NaN,NaN,False
child_healths.infant.following_experience,NaN,NaN,[]


In [286]:
profile = pandas_profiling.ProfileReport(result)
profile

IndexError: index 0 is out of bounds for axis 0 with size 0